##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Entrenar checkpoints

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/checkpoint"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Ver en TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/es/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Ejecutar en Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/es/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Ver fuente en GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/es/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Descargar notebook</a>
  </td>
</table>

Note: Nuestra comunidad de Tensorflow ha traducido estos documentos. Como las traducciones de la comunidad
son basados en el "mejor esfuerzo", no hay ninguna garantia que esta sea un reflejo preciso y actual 
de la [Documentacion Oficial en Ingles](https://www.tensorflow.org/?hl=en).
Si tienen sugerencias sobre como mejorar esta traduccion, por favor envian un "Pull request"
al siguiente repositorio [tensorflow/docs](https://github.com/tensorflow/docs).
Para ofrecerse como voluntario o hacer revision de las traducciones de la Comunidad
por favor contacten al siguiente grupo [docs@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs).

La frase "Saving a TensorFlow model" significa tipicamente una de las dos cosas: (1) Checkpoints, O (2) SavedModel.

Los Checkpoints capturan el valor exacto de todos los parametros (objetos `tf.Variable`) usados por un modelo. Los Checkpoints no almacenan ninguna descripcion del computo utilizado por el modelo. Por lo mismo, los checkpoints solo son utiles cuando el codigo que usara los parametros almacenados esta disponible.

Por otro lado, el formato SavedModel incluye una descripcion serializada del computo definido por el modelo ademas de los valores de los parametros (checkpoint). Con este formato, los modelos son independientes al codigo que creo el mismo. Por ende son idoneos para el despliegue de los modelos a traves de TensorFlos Serving, TensorFlow Lite, TensorFlow.js, o programas en otros lenguajes de programacion (las APIs de TensorFlow para C, C++, Java, Go, Rust, C# etc.)

Esta guia cubre las APIs para leer y escribir checkpoints.

## Guardando de las APIs de entrenamiento de `tf.keras`

Pueden referirse a la [ guia para guardar y restaurar de `tf.keras`](https://www.tensorflow.org/tutorials/keras/save_and_restore_models), NOTA: al momento esta en ingles.

`tf.keras.Model.save_weights` tambien permite la opcion de guardar en el formato TensorFlow checkpoint. Esta guia explica el formato a mayor detalle e introduce las APIs para administrar los checkpoints en bucles de entrenamiento personalizados.

## Definir checkpoints manualmente

El estado persistente de un modelo de TensorFlow es almacenado en objectos `tf.Variable`. Estos objetos pueden ser construidos directamente, pero comunmente con creados mediante APIs de alto nivel tales como `tf.keras.layers`.

La manera mas sencilla de admistrar las variables es asociandolas a objetos de Python, y despues referenciando dichos objetos. Las subclases de `tf.train.Checkpoint`, `tf.keras.layers.Layer`, y `tf.keras.Model` rastrean automaticamente las variables asociadas a sus atributos. El ejemplo a contunuacion construye un modelo linear simple, y posteriormente escribe checkpoints que contienen valores para todas las variables del modelo.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version solo existe en Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

In [0]:
class Net(tf.keras.Model):
  """Un Modelo Linear simple."""

  def __init__(self):
    super(Net, self).__init__()
    self.l1 = tf.keras.layers.Dense(5)

  def call(self, x):
    return self.l1(x)

Este ejemplo necesita datos y un paso de optimizacion para poder ser ejecutable aunque esta guia no se trate de esos temas. El modelo entrenara por slices de un dataset en memoria.

In [0]:
def toy_dataset():
  inputs = tf.range(10.)[:, None]
  labels = inputs * 5. + tf.range(5.)[None, :]
  return tf.data.Dataset.from_tensor_slices(
    dict(x=inputs, y=labels)).repeat(10).batch(2)

In [0]:
def train_step(net, example, optimizer):
  """Entrena `net` en `example` usando `optimizer`."""
  with tf.GradientTape() as tape:
    output = net(example['x'])
    loss = tf.reduce_mean(tf.abs(output - example['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return loss

El siguiente buckle crea una instancia del modelo y de un optimizer, despues los recolecta en un objeto `tf.train.Checkpoint`. Llama el paso de entrenamiento en un ciclo para cada batch de datos, y escribe periodicamente checkpoints en disco.

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)
ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
  print("Restaurado de {}".format(manager.latest_checkpoint))
else:
  print("Inicializando desde cero.")

for example in toy_dataset():
  loss = train_step(net, example, opt)
  ckpt.step.assign_add(1)
  if int(ckpt.step) % 10 == 0:
    save_path = manager.save()
    print("Checkpoint almacenado para el paso {}: {}".format(int(ckpt.step), save_path))
    print("loss {:1.2f}".format(loss.numpy()))

El snippet anterior inicializara aleatoriamente las variables del modelo en su primera corrida. Posterior a esta, reanudara el entrenamiendo en donde se quedo:

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)
ckpt.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
  print("Restaurado de {}".format(manager.latest_checkpoint))
else:
  print("Inicializando desde cero")

for example in toy_dataset():
  loss = train_step(net, example, opt)
  ckpt.step.assign_add(1)
  if int(ckpt.step) % 10 == 0:
    save_path = manager.save()
    print("Checkpoint almacenado para el paso {}: {}".format(int(ckpt.step), save_path))
    print("loss {:1.2f}".format(loss.numpy()))

El objeto `tf.train.CheckpointManager` elimina checkpoints viejos. Arriba ha sido configurado para conservar los tres checkpoints mas recientes.

In [0]:
print(manager.checkpoints)  # Lista los tres checkpoints restantes

Estos paths, e.g. `'./tf_ckpts/ckpt-10'`, no son archivos en disco. Son prefijos para un archivo tipo `index` y uno o mas archivos de datos que contienen los valores de las variables. Estos prefijos estan agrupados en un unico archivo de `checkpoint` (`'./tf_ckpts/checkpoint'`) donde el `CheckpointManager` guarda su estado.

In [0]:
!ls ./tf_ckpts

<a id="loading_mechanics"/>
## Mecanica de carga

TensorFlow hace coincider las variables con los valores almacenados como *checkpoint* atravesando un grafo dirigido con aristas nombradas, comenzando con el objeto que este siendo cargado. Los nombres de las aristas vienen de los nombres de los atributos en los objetos, por ejemplo el `"l1"` en `self.l1 = tf.keras.layers.Dense(5)`. `tf.train.Checkpoint` usa sus argumentos de palabras clave como nombre, tal como el `"step"` en `tf.train.Checkpoint(step=...)`.

El grafo de dependencias del ejemplo anterior se ve asi:

![Visualization of the dependency graph for the example training loop](http://tensorflow.org/images/guide/whole_checkpoint.svg)

Con el *optimizer* en rojo, las variables regulares en azul, y variables slot del *optimizer* en naranja. Los otros nodos, por ejemplo el que representa el `tf.train.Checkpoint`, son negros.

Las variables *Slot* son parte del estado del *optimizer*, pero son creadas para una variable especifica. Por ejemplo las aristas `'m'` de arriba corresponden a un momentum, los cuales son rastreados por el *Adam's* *optimizer* para cada variable. Las variables *Slot* solo son almacenadas en un *checkpoint* si la variable y el optimizer serian ambas almacenadas, por eso los aristas punteados.

La llamada `restore()` de un objeto `tf.train.Checkpoint` hace cola las restauraciones requeridas, restaurando los valores de las variables tan pronto como se encuentre un *path* correspondiente en el objeto `Checkpoint`. Por ejemplo podemos cargar solo el kernel del model que definimos anteriormente recosntruyendo un path a el mediante la red y la capa (layer).

In [0]:
to_restore = tf.Variable(tf.zeros([5]))
print(to_restore.numpy())  # Puros ceros
fake_layer = tf.train.Checkpoint(bias=to_restore)
fake_net = tf.train.Checkpoint(l1=fake_layer)
new_root = tf.train.Checkpoint(net=fake_net)
status = new_root.restore(tf.train.latest_checkpoint('./tf_ckpts/'))
print(to_restore.numpy())  # Ahora obtenemos el valor restaurado

El grafo de dependencias para estos objetos nuevos es un sub-grafo del checkpoint que escribimos anteriormente. Solo incluye el *bias* y un *save counter* que el `tf.train.Checkpoint` usa para enumerar los *checkpoints*.

![Visualization of a subgraph for the bias variable](http://tensorflow.org/images/guide/partial_checkpoint.svg)

`restore()` regresa el estado del objeto, que tiene afirmaciones (assertions) opcionales. Todos los objetos que hemos creado en nuestro nuevo `Checkpoint` han sido restaurados, asi que `status.assert_existing_objects_matched()` pasa exitosamente.

In [0]:
status.assert_existing_objects_matched()

Hay muchos objetos en el *checkpoint* que no han sido emparejados, incluyendo el *kernel* de la capa y las variables del optimized. `status.assert_consumed()` solo pasa si el *checkpoint* y el programa empatan exactamente, y arrojara una excepcion en este caso.

### Restauraciones retrasadas

Los objetos `Layer` en TensorFlow pueden retrasar la creacion de variables para su primera llamada, cuando las dimensiones de entrada estan disponibles. Por ejemplo, las dimensiones de un *layer kernel* `Dense` dependen tanto de las entradas de la capa como de las dimensiones de salida, y por ende solo la dimension de salida que es requerida como argumento de construccion no es suficiente informacion para la creacion de las variables. Como la llamada a `Layer` tambien lee el valor de la variable, una restauracion debe pasa entre las variables de creacion y su primer uso.

Para dar soporte a este idioma, `tf.train.Checkpoint` forma una lista de restauranciones que no tienen una vatiable que empate aun.


In [0]:
delayed_restore = tf.Variable(tf.zeros([1, 5]))
print(delayed_restore.numpy())  # No restaurado; siguen siendo ceros
fake_layer.kernel = delayed_restore
print(delayed_restore.numpy())  # Restaurado

### Revision manual de los checkpoints

`tf.train.list_variables` lista las *checkpoint keys* y las dimensiones de las variables en un checkpoint. Las *Checkpoint keys* son los *paths* del grafo mostrado anteriormente.

In [0]:
tf.train.list_variables(tf.train.latest_checkpoint('./tf_ckpts/'))

### Rastreo de listas y diccionarios
List and dictionary tracking

Igual que en las asignaciones directas de atributos, e.g.  `self.l1 = tf.keras.layers.Dense(5)`, la asignacion de listas y diccionarios a atributos rastreara sus contenidos.

In [0]:
save = tf.train.Checkpoint()
save.listed = [tf.Variable(1.)]
save.listed.append(tf.Variable(2.))
save.mapped = {'one': save.listed[0]}
save.mapped['two'] = save.listed[1]
save_path = save.save('./tf_list_example')

restore = tf.train.Checkpoint()
v2 = tf.Variable(0.)
assert 0. == v2.numpy()  # No ha sido restaurado aun
restore.mapped = {'two': v2}
restore.restore(save_path)
assert 2. == v2.numpy()

Puede notar que existen *wrappers* de objetos para listas y diccionarios. Estos *wrappers* pueden ser incluidos en versiones *checkpoint* de las estructuras de datos subyacientes. Asi como la carga basada en atributos, estos *wrappers* restauran el valor de una variable al momento de ser agregada al contenedor.


In [0]:
restore.listed = []
print(restore.listed)  # ListWrapper([])
v1 = tf.Variable(0.)
restore.listed.append(v1)  # Restaurar v1, del restore() de la celda anterior
assert 1. == v1.numpy()

El mismo rastreo es aplicado automaticamente a subclases de `tf.keras.Model`, y puede ser usado para rastrear listas de capas por ejemplo.


## Guardar checkponts basados en objetos con Estimator

Ver la [guia a Estimator](https://www.tensorflow.org/guide/estimator). NOTA: documentacion en ingles.

Los Estimators guardan checkpoints por default con nombres de variables en lugar de el ogjeto grafo descrito en las secciones anteriores. `tf.train.Checkpoint` aceptara ckeckpoints basadon en nombres, pero los nombres de las variables podrian cambiar al movr las pertes del modelo fuera del `model_fn` del Estimator. Guardar checkpoints basados en objetos facilita el entrenamiento de un modelo dentro de un Estimator y su posterior uso fuera de el.


In [0]:
import tensorflow.compat.v1 as tf_compat

In [0]:
def model_fn(features, labels, mode):
  net = Net()
  opt = tf.keras.optimizers.Adam(0.1)
  ckpt = tf.train.Checkpoint(step=tf_compat.train.get_global_step(),
                             optimizer=opt, net=net)
  with tf.GradientTape() as tape:
    output = net(features['x'])
    loss = tf.reduce_mean(tf.abs(output - features['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  return tf.estimator.EstimatorSpec(
    mode,
    loss=loss,
    train_op=tf.group(opt.apply_gradients(zip(gradients, variables)),
                      ckpt.step.assign_add(1)),
    # Decirle al Estimator gue guarde "ckpt" en un formato basado en objeto.
    scaffold=tf_compat.train.Scaffold(saver=ckpt))

tf.keras.backend.clear_session()
est = tf.estimator.Estimator(model_fn, './tf_estimator_example/')
est.train(toy_dataset, steps=10)

`tf.train.Checkpoint` puede cargar los checkpoints del Estimator de su `model_dir`.

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
ckpt = tf.train.Checkpoint(
  step=tf.Variable(1, dtype=tf.int64), optimizer=opt, net=net)
ckpt.restore(tf.train.latest_checkpoint('./tf_estimator_example/'))
ckpt.step.numpy()  # De est.train(..., steps=10)

## Resumen

Los objetos de TensorFlow proveen un mecanismo facil y automatico para guardar y restaurar los valores de las variables que usan.
